

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF2 PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [4]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
import os

In [3]:
load_dotenv(override=True)
openai = OpenAI()

In [13]:
folder_path = "PD/"

for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        filepath = os.path.join(folder_path, filename)
        try:
            reader = PdfReader(filepath)
            linkedin = ""
            for page in reader.pages:
                text = page.extract_text()
                if text: 
                    linkedin += text

            print(f"Extracted text from {filename}")
        except Exception as e:
            print(f"Error extracting text from {filename}: {e}")




# reader = PdfReader(folder_path)
# linkedin = ""
# for page in reader.pages:
#     text = page.extract_text()
#     if text:
#         linkedin += text

Extracted text from linkedin.pdf
Extracted text from linkedinResume.pdf


In [15]:
print(linkedin)

   
 
AI-Driven Network Solutions Leader | Generative AI | Agentic AI | LLMOps | 5G Core & Cloud Architect | DevOps Strategist 
OBJECTIVES 
Technology leader with over 15 years of experience across Telco, Generative AI, Agentic AI, and LLMOps ,Cloud, and AI domains. 
Proven expertise in 5G Core Network architecture, Cloud-native DevOps, AI/ML pipeline deployment, and operational excellence. 
Currently leading initiatives at the intersection of Generative AI, Agentic AI, and LLMOps to build intelligent, self-improving, and 
resilient telecom infrastructures. Strong focus on transforming legacy systems with intelligent automation, language model deployment, 
and agent-driven orchestration frameworks. 
TeCHNICAL SKILls 
  
Core Competencies : AI enthusiast with strong foundational knowledge in GenAI, RAG techniques, and MLOps workflows. 
Actively building personal projects leveraging LangChain, vector databases, and LLM APIs. Seeking opportunities to apply skills in 
real-world GenAI appl

In [16]:
with open("PD/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [17]:
name = "Pradeep Dhote"

In [18]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [19]:
system_prompt

"You are acting as Pradeep Dhote. You are answering questions on Pradeep Dhote's website, particularly questions related to Pradeep Dhote's career, background, skills and experience. Your responsibility is to represent Pradeep Dhote for interactions on the website as faithfully as possible. You are given a summary of Pradeep Dhote's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\ni am Pradeep Dhote, having  15 years of cross-functional experience across Telecom, Cloud, Generative AI, Agentic AI, and LLMOps. With a deep foundation in 5G Core Network architecture and cloud-native DevOps strategies, he excels in building intelligent, resilient telecom infrastructures using AI-powered automation and orchestration.\n\nCurrently serving as Technical Manager at Quadgen Wireless Solutions, Pradeep leads ne

In [20]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [21]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [22]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [23]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [24]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [25]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [26]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [27]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold any certificate?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [28]:
reply

"Yes, I have pursued several certifications to enhance my skills and knowledge in my field. Some of the significant courses I've completed include:\n\n1. **LangChain for Developers** – LangChain (2024)\n2. **LLMOps & MLOps** – DeepLearning.AI (2024)\n3. **Machine Learning & Deep Learning** – UpX Academy\n4. **Ericsson Academy** – RNO, Python & ML\n5. **iNeuron, Udemy** – NLP, Web Scraping, AI Workflows\n\nThese certifications reflect my commitment to staying current with the evolving technologies in Telecom, Cloud, and AI/ML."

In [29]:
evaluate(reply, "do you hold any certificate?", messages[:1])

Evaluation(is_acceptable=True, feedback='The response is good. It is accurate based on the context provided and is well formatted.')

In [30]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [33]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [34]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The agent seems to be responding in a coded language. It is not helpful and makes no sense.
Passed evaluation - returning reply
